In [8]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


# Creating dummies

In [10]:
# Copy the dataframe
df2 = data.copy()

In [11]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [12]:
df2.head(20)

,ID,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Country,...,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023,China
8,1,0.010373,Aiways,U5,400,284621.700,201,34,SUV,CH,...,0,0,0,0,0,0,1,0,0,1
9,1,0.005976,Aiways,U5,400,313681.829,201,34,SUV,CH,...,0,0,0,0,0,0,0,1,0,1
10,1,0.002860,Aiways,U5,400,264524.000,201,34,SUV,CH,...,0,0,0,0,0,0,0,0,1,1
21,2,0.000048,Aiways,U6,405,360638.000,214,34,SUV,CH,...,0,0,0,0,0,0,0,0,1,1
28,3,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,DE,...,0,0,0,0,1,0,0,0,0,0
29,3,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,DE,...,0,0,0,0,0,1,0,0,0,0
30,3,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,DE,...,0,0,0,0,0,0,1,0,0,0
31,3,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,DE,...,0,0,0,0,0,0,0,1,0,0
32,3,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,DE,...,0,0,0,0,0,0,0,0,1,0
41,4,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,DE,...,0,0,0,0,0,0,1,0,0,0


# Creating log market share

In [13]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# Independent OLS

In [14]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'Price', 'HP', 'Chargetime']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)

In [15]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit()
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     14.19
Date:                Wed, 06 Mar 2024   Prob (F-statistic):           4.00e-35
Time:                        21:25:52   Log-Likelihood:                -639.15
No. Observations:                 334   AIC:                             1324.
Df Residuals:                     311   BIC:                             1412.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6280      0.955     -3.800      0.0